In [ ]:
import pandas as pd
from kiwipiepy import Kiwi

merged_df = pd.read_csv('/content/drive/MyDrive/asia/merged_df.csv')

# 'description'과 'requirement' 열 합치기 및 문자열로 변환
merged_df['combined_text'] = merged_df[['description', 'requirement']].fillna('').astype(str).agg(' '.join, axis=1)

# 'preferredExperience' 열 추출
merged_df['preferredExperience'] = merged_df['preferredExperience'].fillna('')

# Kiwi 초기화
kiwi = Kiwi(typos='basic_with_continual_and_lengthening')

# 불용어 설정
stop_words = kiwi.utils.Stopwords()

#  분석 결과에 등장하면 안되는 형태소 목록을 지정
# blocklist = ['고마움/NNG], stopwords.add

# kiwi.add_user_word('대학생 선교회', 'NNP'), kiwi.add_user_word('C++', 'NNP')

# split_complex 더 잘게 분할 가능한 형태소들을 최대한 분할 ex)'고마움(고맙 + 음)'과 같은 파생 명사, '건강히(건강 + 히)'와 같은 파생 부사, '반짝거리다(반짝 + 거리다)', '걸어다니다(걸어 + 다니다)'

# 형태소 분석 함수
def analyze_text(text):
    if not isinstance(text, str):
        return []
    morphemes_kiwi = kiwi.tokenize(text, normalize_coda=True, stopwords=stop_words, split_complex=True)
    return [morph for morph, pos, _, _ in morphemes_kiwi if pos.startswith('N')]

def analyze_text_all(text):
    if not isinstance(text, str):
        return []
    morphemes_kiwi = kiwi.tokenize(text, normalize_coda=True, stopwords=stop_words, split_complex=True)
    return [(morph, pos) for morph, pos, _, _ in morphemes_kiwi]

# 형태소 분석 적용
merged_df['combined_text_analyzed'] = merged_df['combined_text'].apply(analyze_text)
merged_df['preferredExperience_analyzed'] = merged_df['preferredExperience'].apply(analyze_text)

merged_df['combined_text_analyzed_all'] = merged_df['combined_text'].apply(analyze_text_all)
merged_df['preferredExperience_analyzed_all'] = merged_df['preferredExperience'].apply(analyze_text_all)

# 결과를 새로운 CSV 파일에 저장
merged_df[['id', 'description', 'requirement', 'preferredExperience', 'combined_text_analyzed', 'combined_text_analyzed_all', 'preferredExperience_analyzed', 'preferredExperience_analyzed_all']].to_csv('analyzed_job_data.csv', index=False, encoding = 'utf-8-sig')
